In [10]:
import os
import numpy as np
from ast import literal_eval as make_tuple


directory = "."

ds_names = {"Reynolds","UK"}
all_models = []
dat = {} 
for file in os.listdir(directory):
    if file.endswith(".txt"): 
        with open(os.path.join(directory, file),"r") as f:
            for line in f:
                if line.find("y_") > -1:
                    if line.find("(") > -1 and line.find(")") > -1:
                        tpl = make_tuple(line)    
                        line = " ".join([str(t) for t in tpl])    
                    
                    for dname in ds_names:
                        start = line.find(dname)
                        if start > -1:
                            model_name = line[:start].strip()
                            all_models.append(model_name)
                            res = line[start + len(dname):].strip().split()

                            deep = res[0]
                            r2 = res[2]
                            r1 = res[1]
                            dat.setdefault(dname,{}).setdefault(deep,{}).setdefault(model_name,[r1,r2])

all_models = set(all_models)
new_dat = []

rtable = {"y_1":"5","y_2":"10","y_3":"20","y_4":"30","y_5":"40","y_6":"50","y_7":"60","y_8":"90"}
utable = {"y_1":"10","y_2":"10","y_3":"5","y_4":"5","y_5":"15","y_6":"15","y_7":"25","y_8":"25","y_9":"50","y_10":"50"}

for d in dat:
    for dep in dat[d]:
        if d == "Reynolds":
            dp =  rtable[dep]   
        else:
            if d == "UK":
                dp = utable[dep]
                
        tmp = [d, dp]
        for model in all_models:
            if model in dat[d][dep]:
                tmp += [np.sqrt(float(dat[d][dep][model][0])), float(dat[d][dep][model][1])]
            else:
                tmp += [None,None]
        new_dat.append(tmp)    


                        

In [11]:
import pandas as pd
from functools import partial

def bold_formatter(x, values, num_decimals=2):
    match = False
    for v in values:
         if round(x, num_decimals) == round(v, num_decimals):
             match = True
             break
             
    if match:
        return f"\\textbf{{{x:.{num_decimals}f}}}"
    else:
        return f"{x:.{num_decimals}f}"

pd.options.display.float_format = '{:.2f}'.format

names_top = [model for model in all_models for _ in (0, 1)]
names_bottom = [i for _ in all_models for i in ("RMSE", "MAE")]

df = pd.DataFrame(new_dat, columns=[["",""] + names_top, ["Dataset","Depth, cm"] + names_bottom])

for i,c in enumerate(df.columns):
    if i > 1:
        df[c] = pd.to_numeric(df[c])

df = df.set_index([("","Dataset"),("","Depth, cm")]).sort_index()

col2_bold = {}

for i,r in df.iterrows():
    ma_odd = r[0::2].min() 
    ma_ev =  r[1::2].min()
    ma_odd_id = 2 * r[0::2].argmin()
    ma_ev_id = 2 * r[1::2].argmin() + 1
    ma_odd = r[ma_odd_id]
    ma_ev = r[ma_ev_id]
    if not df.columns[ma_odd_id] in col2_bold:
        col2_bold[df.columns[ma_odd_id]] = []
    col2_bold[df.columns[ma_odd_id]].append(ma_odd)
    if not df.columns[ma_ev_id] in col2_bold:
        col2_bold[df.columns[ma_ev_id]] = []
    col2_bold[df.columns[ma_ev_id]].append(ma_ev)
    
fmts_max_2f = {column: partial(bold_formatter, values=col2_bold[column], num_decimals=2) for column in col2_bold}



/tmp/ipykernel_116/99201611.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ma_odd = r[ma_odd_id]
/tmp/ipykernel_116/99201611.py:37: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ma_ev = r[ma_ev_id]


In [12]:
print(df.to_latex(float_format="%.2f",formatters=fmts_max_2f))

\begin{tabular}{llrrrrrrrrrrrrrrrrrr}
\toprule
 &  & \multicolumn{2}{r}{LSTM} & \multicolumn{2}{r}{ecdfr} & \multicolumn{2}{r}{Boosted Forest} & \multicolumn{2}{r}{ARIMA} & \multicolumn{2}{r}{XGB} & \multicolumn{2}{r}{TKAN} & \multicolumn{2}{r}{Cascade Forest} & \multicolumn{2}{r}{AWDF} & \multicolumn{2}{r}{GRU} \\
 &  & RMSE & MAE & RMSE & MAE & RMSE & MAE & RMSE & MAE & RMSE & MAE & RMSE & MAE & RMSE & MAE & RMSE & MAE & RMSE & MAE \\
('', 'Dataset') & ('', 'Depth, cm') &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{8}{*}{Reynolds} & 10 & 2.08 & 1.59 & 2.51 & 1.84 & 1.57 & 1.06 & \textbf{1.45} & \textbf{0.99} & 1.65 & 1.15 & 2.66 & 1.65 & 2.92 & 2.09 & 2.92 & 2.26 & 1.89 & 1.52 \\
 & 20 & 1.78 & 1.39 & 2.05 & 1.49 & \textbf{1.01} & \textbf{0.67} & 1.25 & 0.73 & 1.14 & 0.80 & 1.25 & 0.91 & 2.33 & 1.65 & 1.70 & 1.30 & 1.95 & 1.40 \\
 & 30 & 0.99 & 0.80 & 1.49 & 1.05 & \textbf{0.55} & \textbf{0.33} & 1.01 & 0.49 & 0.68 & 0.46 & 2.90 & 2.04 & 1.62 & 1.09 &

In [13]:
df

LSTM      ecdfr      Boosted Forest      ARIMA       \
                          RMSE  MAE  RMSE  MAE           RMSE  MAE  RMSE  MAE   
(, Dataset) (, Depth, cm)                                                       
Reynolds    10            2.08 1.59  2.51 1.84           1.57 1.06  1.45 0.99   
            20            1.78 1.39  2.05 1.49           1.01 0.67  1.25 0.73   
            30            0.99 0.80  1.49 1.05           0.55 0.33  1.01 0.49   
            40            0.92 0.74  1.58 1.15           0.53 0.37  0.74 0.47   
            5             3.42 2.72  2.96 2.00           1.89 1.13  1.97 1.24   
            50            1.57 1.20  1.41 1.02           0.37 0.25  0.57 0.35   
            60            1.18 0.89  1.36 1.03           0.37 0.21  0.37 0.22   
            90            1.38 1.05  0.95 0.76           0.16 0.12  0.09 0.07   
UK          10            0.78 0.49  1.02 0.75           0.52 0.38  0.73 0.47   
            10            0.84 0.62  1.14 0.84           0.54 0.38  0.66 0.45   
            15            0.68 0.51  1.04 0.76           0.45 0.33  0.61 0.42   
            15            0.75 0.53  1.05 0.76            NaN  NaN  0.61 0.44   
            25            0.75 0.59  0.91 0.66           0.29 0.21  0.38 0.28   
            25            0.99 0.72  1.11 0.86           0.31 0.21  0.41 0.29   
            5             0.87 0.66  1.31 0.97           0.71 0.52  0.90 0.65   
            5             1.15 0.75  1.20 0.86           0.78 0.54  0.88 0.64   
            50            0.57 0.47  0.81 0.60           0.25 0.16  0.25 0.16   
            50            0.79 0.58  0.98 0.77            NaN  NaN   NaN  NaN   

                           XGB      TKAN      Cascade Forest      AWDF       \
                          RMSE  MAE RMSE  MAE           RMSE  MAE RMSE  MAE   
(, Dataset) (, Depth, cm)                                                     
Reynolds    10            1.65 1.15 2.66 1.65           2.92 2.09 2.92 2.26   
            20            1.14 0.80 1.25 0.91           2.33 1.65 1.70 1.30   
            30            0.68 0.46 2.90 2.04           1.62 1.09 1.28 0.93   
            40            0.71 0.53 0.63 0.50           1.85 1.37 1.59 1.24   
            5             1.94 1.17 2.16 1.44           3.12 2.11 3.84 2.87   
            50            0.57 0.40 0.41 0.30           1.73 1.25 1.36 1.05   
            60            0.52 0.37 0.27 0.19           1.54 1.18 1.32 1.00   
            90            0.35 0.27 0.27 0.17           1.22 0.90 1.35 1.14   
UK          10            0.61 0.45 1.89 1.30           1.40 0.91 1.50 0.97   
            10            0.64 0.46 0.70 0.46           1.43 0.94 1.64 1.06   
            15            0.55 0.41 0.56 0.42           1.16 0.80 1.18 0.78   
            15            0.58 0.44 1.45 0.93           1.32 0.85 1.31 0.93   
            25            0.40 0.30 0.82 0.54           1.02 0.70 1.05 0.73   
            25            0.41 0.31 4.49 3.79           1.13 0.72 1.10 0.75   
            5             0.84 0.63 1.58 1.11           1.40 0.98 1.41 0.98   
            5             0.85 0.62 1.66 1.10           1.51 1.00 1.63 1.00   
            50            0.33 0.23 1.07 0.84           1.06 0.73 1.16 0.90   
            50            0.35 0.24 1.40 0.99           0.99 0.70 1.07 0.81   

                           GRU       
                          RMSE  MAE  
(, Dataset) (, Depth, cm)            
Reynolds    10            1.89 1.52  
            20            1.95 1.40  
            30            2.41 1.81  
            40            0.99 0.76  
            5             4.26 3.19  
            50            0.76 0.59  
            60            1.55 1.17  
            90            1.92 1.50  
UK          10            1.02 0.85  
            10            0.97 0.63  
            15            0.73 0.54  
            15            0.79 0.58  
            25            0.68 0.49  
            25            0.74 0.49  
            5     